<a href="https://colab.research.google.com/github/ArtuomMerkulov/ArtuomMerkulov/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22text_generation_fnet%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filename = "/content/drive/MyDrive/Numpy/philosophy_dataset.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [ ]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(char_to_int)

{'\n': 0, '\x0c': 1, ' ': 2, '!': 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, ':': 9, ';': 10, '?': 11, 'c': 12, '«': 13, '»': 14, 'а': 15, 'б': 16, 'в': 17, 'г': 18, 'д': 19, 'е': 20, 'ж': 21, 'з': 22, 'и': 23, 'й': 24, 'к': 25, 'л': 26, 'м': 27, 'н': 28, 'о': 29, 'п': 30, 'р': 31, 'с': 32, 'т': 33, 'у': 34, 'ф': 35, 'х': 36, 'ц': 37, 'ч': 38, 'ш': 39, 'щ': 40, 'ъ': 41, 'ы': 42, 'ь': 43, 'э': 44, 'ю': 45, 'я': 46, 'ё': 47, '–': 48, '—': 49, '…': 50}


In [ ]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  21016
Total Vocab:  51


In [ ]:
seq_length = 1000
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  20016


In [ ]:
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [24]:
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
#model.add(LSTM(256))
#model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [25]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [26]:
model.fit(X, y, epochs=100, batch_size=256, callbacks=callbacks_list)

Epoch 1/100
79/79 [==============================] - ETA: 0s - loss: 3.2961
Epoch 1: loss improved from inf to 3.29612, saving model to weights-improvement-01-3.2961.hdf5
79/79 [==============================] - 46s 560ms/step - loss: 3.2961
Epoch 2/100
79/79 [==============================] - ETA: 0s - loss: 3.2194
Epoch 2: loss improved from 3.29612 to 3.21944, saving model to weights-improvement-02-3.2194.hdf5
79/79 [==============================] - 46s 584ms/step - loss: 3.2194
Epoch 3/100
79/79 [==============================] - ETA: 0s - loss: 3.2135
Epoch 3: loss improved from 3.21944 to 3.21348, saving model to weights-improvement-03-3.2135.hdf5
79/79 [==============================] - 45s 568ms/step - loss: 3.2135
Epoch 4/100
79/79 [==============================] - ETA: 0s - loss: 3.2106
Epoch 4: loss improved from 3.21348 to 3.21062, saving model to weights-improvement-04-3.2106.hdf5
79/79 [==============================] - 45s 574ms/step - loss: 3.2106
Epoch 5/100
79/79 [=

KeyboardInterrupt: ignored

In [27]:
# load the network weights
filename = "weights-improvement-72-0.0591.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [28]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [29]:
import sys

In [30]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Read:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Read:
"  живет в согласии со всеми, а низкий человек ищет себе подобных.
благородный знает о своем превосходстве, но избегает соперничества. он ладит со всеми, но ни с кем не
вступает в сговор.
благородный муж прям и тверд, но не упрям.
благородный муж стремится говорить косноязычно, а действовать искусно.
благородный не стремится есть досыта и жить богато. он поспешает в делах, но медлит в речах. общаясь
с людьми добродетельными, он исправляет себя. вот о таком человеке можно сказать, что он предан
учению.
благородный ни от кого не ожидает обмана, но когда его обманывают, он первый замечает это.
благородный помогает людям увидеть доброе в себе и не поучает людей видеть в себе дурное. а низкий
человек поступает наоборот.
благородный превыше всего почитает долг. благородный человек, наделенный отвагой, но не ведающий
долга, может стать мятежником. низкий человек, наделенный отвагой, но не ведающий долга, может
пуститься в разбой.
благородный с достоинством ожидает велений неба. низкий ч